# Brain teaser

[Original Github REPO](https://github.com/1171-jpg/BrainTeaser)


### Upute za pokretanje

1) Pokrenuti donju liniju i viditi jel se sve moze importati
1) Skinuti sve sto se ne moze sa `!pip install` unutar notebooka
(pip bi trebao raditi i s condom ako je dobro namjesteno)
    - Skinuti pytorch sa stranice [Pytorch](https://pytorch.org/get-started/locally/) - odaberite CUDA 11.8
    ako imate graficku. Meni je bilo:
    `> !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118`
    - Ako nemate CUDA, notebook ce biti sporiji - dovoljan za inference, 
    ali nedovoljan za fine-tuning (sreca pa to niti ne radimo, bilo bi presporo 
    i s grafickom)
    - Skinuti huggingface transformers library:
    `> pip install 'transformers[torch]'`
1) Moze se sve importati na Kaggle, ovo mozda malo kasnije

In [1]:
import numpy as np
import json
from datasets import load_dataset
from random import shuffle
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

from tqdm.auto import tqdm
import numpy as np
import os
from transformers import StoppingCriteriaList

In [127]:
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'][:12]
letters, len(letters)

(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L'], 12)

In [128]:
import json
from tqdm import tqdm
import torch
import logging
import argparse
import numpy as np

def getResultdata(result_data):
    choice_to_index = {letters[i]: i for i in range(len(letters))}
    choice_to_index[None] = len(letters)

    word_play = {}
    reverse_play = {}
    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        if item_type == 'WP':
            if item_id not in word_play:
                word_play[item_id] = [0,0,0]
        else:
            if item_id not in reverse_play:
                reverse_play[item_id] = [0,0,0]

    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        ad_type = 0
        if 'SR' in item['id']:
            ad_type = 1
        elif 'CR' in item['id']:
            ad_type = 2
        else:
            ad_type = 0

        if item_type == 'WP':
            if choice_to_index[item['predict']] == item['label']:
                word_play[item_id][ad_type] = 1
        else:
            if choice_to_index[item['predict']] == item['label']:
                reverse_play[item_id][ad_type] = 1
                
    return word_play,reverse_play


def getMetric(data_list):
    data_list = np.array(data_list)
    overall_accuracy = np.sum(data_list)/3/len(data_list)
    original_accuracy = np.sum(data_list,axis = 0)[0]/len(data_list)
    semantic_accuracy = np.sum(data_list,axis = 0)[1]/len(data_list)
    context_accuracy = np.sum(data_list,axis = 0)[2]/len(data_list)
    ori_sema = np.sum([1 if item[0]==1 and item[1] == 1 else 0 for item in data_list])/len(data_list)
    ori_sema_cont = np.sum([1 if item[0]==1 and item[1] == 1 and item[2] == 1  else 0 for item in data_list])/len(data_list)
    
    print("over_all accuracy {}".format(overall_accuracy))
    print("single_original_accuracy {}".format(original_accuracy))
    print("single_semantic_accuracy {}".format(semantic_accuracy))
    print("single_context_accuracy {}".format(context_accuracy))
    print("sr_accuracy {}".format(ori_sema))
    print("cr_accuracy {}".format(ori_sema_cont))

    return {'over_all accuracy':overall_accuracy,'original_accuracy':original_accuracy,'semantic_accuracy':semantic_accuracy,'context_accuracy':context_accuracy,'ori_sema':ori_sema,'ori_sema_cont':ori_sema_cont}


def getSeperateResult(word_play,reverse_thinking):
    final_result = {}
    word_data_list = []
    word_data_list = list(word_play.values())
    print('#########Wordplay##########')
    final_result['wordplay'] = getMetric(word_data_list)
    
    reverse_data_list = []
    for item in reverse_thinking.values():
        reverse_data_list.append(item)
    print('#########Sentence##########')   
    final_result['sentence'] = getMetric(reverse_data_list)  
    
    
    all_data = word_data_list + reverse_data_list
    print('#########All data##########') 
    final_result['all'] = getMetric(all_data) 
    
    return final_result

In [129]:
def save(name=None, few_shot=False, example=format):
    try:
        os.mkdir('results')
    except:
        pass
    model_name = model_path.split('/')[-1]
    inserted_name = "" if name is None else "_" + name
    i = 1
    while True:
        file_path = f'results/{model_name}{inserted_name}_{i}.txt'
        try:
            with open(file_path, 'r'):
                i += 1
        except:
            with open(file_path, 'w') as file:
                file.write(model_name+'\n')
                if few_shot:
                    file.write(few_shot + '\n')
                if not isinstance(example, str):
                    example = json.dumps(example)
                file.write(example + '\n')
                file.write(json.dumps(final_result, indent=4))
                return

### Dataset

- Skinuti s [linka](https://drive.google.com/drive/u/0/folders/1kiFXp5fqpf8--NQJJAlBIBpfSaXTk1UY)

- Staviti u folder `/data`

In [130]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Load the model

In [131]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
# model_path = 'google/flan-t5-xxl'
# model_path = 'google/flan-t5-xl'
# model_path = 'google/flan-t5-large'
# model_path = 'mistralai/Mistral-7B-Instruct-v0.2'
model_path = 'microsoft/phi-2'

tokenizer = AutoTokenizer.from_pretrained(model_path)

if model_path.find('flan') >= 0:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
else:
    model = AutoModelForCausalLM.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [132]:
try:
    model = model.to(device)
except Exception as e:
    print(e)
    print("Changing device to cpu")
    device = 'cpu'

In [133]:
# few_shot_examples = list(np.load("./{dataset_path}/data/demonstration.npy",allow_pickle=True))
data_path = '/kaggle/input/brainteaser/data'
sentence_data_path = f"{data_path}/SP-train.npy"
wordplay_data_list = f"{data_path}/WP-train.npy"
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

In [134]:
test_data_list = sentence_data_list + wordplay_data_list
print(f"Dataset length {len(test_data_list)}")

Dataset length 903


In [135]:
newline = '\n'
format = f"""Question: {"{}"}
Choice:
{''.join('(' + x + ') {}' + newline for x in letters)}
Answer:("""
print(format)

Question: {}
Choice:
(A) {}
(B) {}
(C) {}
(D) {}
(E) {}
(F) {}
(G) {}
(H) {}
(I) {}
(J) {}
(K) {}
(L) {}

Answer:(


In [136]:
def get_single_demo(sample):
    sample_demo = format.format(
        sample['question'], *sample['choice_list'])
    return sample_demo

In [137]:
good_responses = [f'{x})' for x in letters]
good_responses

['A)', 'B)', 'C)', 'D)', 'E)', 'F)', 'G)', 'H)', 'I)', 'J)', 'K)', 'L)']

In [138]:
def custom_stopping_criteria(input_ids: torch.LongTensor, score: torch.FloatTensor, **kwargs) -> bool:
    decoded = tokenizer.decode(input_ids[0][-3:])
    for good_response in good_responses:
        if good_response in decoded:
            return True
    return False

stopping_criteria = StoppingCriteriaList([custom_stopping_criteria])

In [139]:
def set_predictions(data_list):
    for index,item in enumerate(data_list):
        item['predict'] = None
        for x in letters:
            if (f'{x})') in item['response']:
                item['predict'] = x

        if item['predict'] is None:
            print(index)

In [140]:
from collections import defaultdict

def generate(samples, tokens=10, all_tokens=True, few_shot=False):
    for sample in tqdm(samples):
        if few_shot:
            text = demonstration + get_single_demo(sample)
        else:
            text = get_single_demo(sample)
        inputs = tokenizer.encode(text, return_tensors="pt", return_attention_mask=False).to(device)
        original_tokens = len(inputs[0])
        outputs = model.generate(
            inputs,
            pad_token_id=tokenizer.eos_token_id,
            do_sample = False,
            max_new_tokens=tokens,
            stopping_criteria=stopping_criteria
        )
        outputs = outputs[0][0 if all_tokens else original_tokens:]
        sample['response'] = tokenizer.decode(outputs)
    set_predictions(samples)

In [141]:
def create_with_question(i):
    current = test_data_list[i]
    new = {**current}
    choice_list = [*current['choice_list']]
    choice_list[current['choice_order'].index(2)] = current['question'][:-1]
    new['choice_list'] = choice_list
    return new

def test_with_question(generated_data_list):
    for i, sample in enumerate(generated_data_list):
        if i == 755: continue
        assert sample['distractor2'] not in sample['choice_list'], i
        assert sample['question'][:-1] in sample['choice_list'], i

def create_with_k_questions(i, k):
    current = test_data_list[i]
    choice_list = [*current['choice_list']]
    choice_order = [*current['choice_order']]
    while len(choice_list) < k-1:
        i -= 11
        previous = test_data_list[i]
        choice_list.insert(-1, previous['distractor1'])
        choice_list.insert(-1, previous['distractor2'])
    last_order = choice_list[-1]
    choice_list = choice_list[:-1]
#     print(current['answer'])
    shuffle(choice_list)
    choice_list.append(last_order)
    try:
        label = choice_list.index(current['answer'])
    except Exception as e:
        print(current['answer'])
        label = len(choice_list)-1
    new = {**current}
    new['label'] = label
    new['choice_list'] = choice_list
    return new

def test_with_k_questions(generated_data_list):
    for original, new in zip(test_data_list, generated_data_list):
        assert set(original['choice_list']).issubset(set(new['choice_list'])), str(original) + '\n'+ str(new)

random.seed(10)
generated_data_list = []
for i in range(len(test_data_list)):
    new = create_with_k_questions(i, len(letters))
    generated_data_list.append(new)
test_with_k_questions(generated_data_list)

They all do.
None of above. No dogs left the yard.
None of above. No dogs left the yard.
None of above. No birds left the branch.


In [142]:
generated_data_list[0]
for sample in generated_data_list:
    if sample['id'] == 'SP-63':
        display(sample)

{'id': 'SP-63',
 'question': 'Which would see most clearly in total darkness? A bat, a tiger, or an owl.',
 'answer': 'None of above.',
 'distractor1': 'Tiger.',
 'distractor2': 'Bat.',
 'distractor(unsure)': 'Owl.',
 'label': 11,
 'choice_list': ['The glass is too heavy.',
  'He uses a computer to compute his actions.',
  'He told Danny to left the bridge.',
  'Surface tension prevents water from spilling.',
  'Tiger.',
  'He uses the sound to feel the direction.',
  'His husband is not a good husband.',
  'Owl.',
  'Bat.',
  'He told Danny to overturn the track.',
  'The woman is not a good person.',
  'None of above.'],
 'choice_order': [3, 1, 2, 0]}

In [143]:
generate(generated_data_list[:10], tokens=1000, all_tokens=False, few_shot=False)

100%|██████████| 10/10 [00:02<00:00,  3.80it/s]


In [144]:
for sample in generated_data_list[:5]:
    print("\n>", sample['response'])


> J)

> E)

> B)

> I)

> A)


In [145]:
generate(generated_data_list, tokens=20, all_tokens=False, few_shot=False)

100%|██████████| 903/903 [03:52<00:00,  3.88it/s]


In [146]:
# generate(test_data_list, tokens=20, all_tokens=False, few_shot=False)

In [147]:
word_play,sentence_play = getResultdata(generated_data_list)
# word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play, sentence_play)

#########Wordplay##########
over_all accuracy 0.17424242424242425
single_original_accuracy 0.14393939393939395
single_semantic_accuracy 0.16666666666666666
single_context_accuracy 0.21212121212121213
sr_accuracy 0.06818181818181818
cr_accuracy 0.03787878787878788
#########Sentence##########
over_all accuracy 0.28402366863905326
single_original_accuracy 0.27218934911242604
single_semantic_accuracy 0.24260355029585798
single_context_accuracy 0.33727810650887574
sr_accuracy 0.14201183431952663
cr_accuracy 0.07692307692307693
#########All data##########
over_all accuracy 0.23588039867109634
single_original_accuracy 0.2159468438538206
single_semantic_accuracy 0.20930232558139536
single_context_accuracy 0.2823920265780731
sr_accuracy 0.10963455149501661
cr_accuracy 0.059800664451827246


In [148]:
save(name="12_answers", example=get_single_demo(generated_data_list[0]))

In [67]:
# save(few_shot=demonstration)
# save()